## 🚀 PIPELINE MULTIFORMATO - DWG & RVT

### BLOCO 1: CONFIGURAÇÃO INICIAL E DEPENDÊNCIAS

In [48]:
# =============================================================================
# 1. CONFIGURAÇÃO INICIAL E DEPENDÊNCIAS
# =============================================================================

print("🚀 INICIANDO PIPELINE MULTIFORMATO")
print("=" * 50)

# Configurações básicas
import pandas as pd
import numpy as np
from pathlib import Path
import sys
from datetime import datetime
import json
import re
from collections import Counter

# Verificar dependências multimídia
def verificar_dependencias():
    """Verifica e instala dependências para novos formatos"""
    
    dependencias = {
        'PyPDF2': 'PDF',
        'dxfgrabber': 'DWG',
        'ezdxf': 'DWG', 
        'pythonnet': 'RVT',
        'clr': 'RVT'
    }
    
    print("🔍 VERIFICANDO DEPENDÊNCIAS MULTIFORMATO")
    print("-" * 40)
    
    status = {}
    
    for lib, formato in dependencias.items():
        try:
            if lib == 'clr':
                import clr
            else:
                __import__(lib)
            status[lib] = True
            print(f"✅ {lib:15} | Suporte {formato:4} | Disponível")
        except ImportError:
            status[lib] = False
            print(f"❌ {lib:15} | Suporte {formato:4} | Não instalado")
    
    return status

# Executar verificação
status_deps = verificar_dependencias()

# Configurar flags de suporte
PDF_SUPORTADO = status_deps.get('PyPDF2', False)
DWG_SUPORTADO = status_deps.get('dxfgrabber', False) and status_deps.get('ezdxf', False)
RVT_SUPORTADO = status_deps.get('pythonnet', False) and status_deps.get('clr', False)

print(f"\n📊 STATUS DOS FORMATOS:")
print(f"{'✅' if PDF_SUPORTADO else '❌'} PDF  - {'Totalmente suportado' if PDF_SUPORTADO else 'Dependências necessárias'}")
print(f"{'✅' if DWG_SUPORTADO else '❌'} DWG - {'Totalmente suportado' if DWG_SUPORTADO else 'Dependências necessárias'}")
print(f"{'✅' if RVT_SUPORTADO else '❌'} RVT - {'Totalmente suportado' if RVT_SUPORTADO else 'Dependências necessárias'}")

🚀 INICIANDO PIPELINE MULTIFORMATO
🔍 VERIFICANDO DEPENDÊNCIAS MULTIFORMATO
----------------------------------------
✅ PyPDF2          | Suporte PDF  | Disponível
✅ dxfgrabber      | Suporte DWG  | Disponível
✅ ezdxf           | Suporte DWG  | Disponível
✅ pythonnet       | Suporte RVT  | Disponível
✅ clr             | Suporte RVT  | Disponível

📊 STATUS DOS FORMATOS:
✅ PDF  - Totalmente suportado
✅ DWG - Totalmente suportado
✅ RVT - Totalmente suportado


### BLOCO 2: INSTALAÇÃO DE DEPENDÊNCIAS

In [49]:
# =============================================================================
# 2. INSTALAÇÃO DE DEPENDÊNCIAS (OPCIONAL)
# =============================================================================

print("\n📦 INSTALAÇÃO DE DEPENDÊNCIAS")
print("=" * 30)

print("💡 COMANDOS PARA INSTALAÇÃO:\n")

if not DWG_SUPORTADO:
    print("Para DWG (AutoCAD):")
    print("pip install dxfgrabber ezdxf")
    print()

if not RVT_SUPORTADO:
    print("Para RVT (Revit):")
    print("pip install pythonnet")
    print()

print("⚠️  Nota: O processamento RVT requer Revit instalado no sistema")


📦 INSTALAÇÃO DE DEPENDÊNCIAS
💡 COMANDOS PARA INSTALAÇÃO:

⚠️  Nota: O processamento RVT requer Revit instalado no sistema


### BLOCO 3: PROCESSADOR DWG (AUTOCAD)

In [50]:
# =============================================================================
# 3. PROCESSADOR DWG (AUTOCAD)
# =============================================================================

class ProcessadorDWG:
    """Processador para arquivos DWG (AutoCAD)"""
    
    def __init__(self):
        self.entidades_relevantes = [
            'LINE', 'CIRCLE', 'ARC', 'TEXT', 'MTEXT', 
            'INSERT', 'DIMENSION', 'LWPOLYLINE', 'LAYER'
        ]
        self.padroes_materiais = self._criar_padroes_materiais()
    
    def _criar_padroes_materiais(self):
        """Padrões regex para identificar materiais em textos DWG"""
        return {
            'codigo_descricao': r'(M\\d{2,4})\\s*[-–]?\\s*([^-–]+?)',
            'dimensoes': r'(\\d+)\\s*x\\s*(\\d+)(?:\\s*x\\s*(\\d+))?\\s*(cm|m|mm)',
            'quantidade': r'(\\d+)\\s*(un|UN|pç|pc|und|m²|m³)'
        }
    
    def extrair_dados(self, dwg_path):
        """Extrai dados de arquivo DWG"""
        print(f"   🔧 Processando DWG: {dwg_path.name}")
        
        if not DWG_SUPORTADO:
            return {'erro': 'Dependências DWG não disponíveis'}
        
        try:
            import dxfgrabber
            
            # Ler arquivo DWG
            dwg = dxfgrabber.readfile(str(dwg_path))
            
            dados_extraidos = {
                'metadados': self._extrair_metadados(dwg),
                'entidades': self._extrair_entidades(dwg),
                'textos': self._extrair_textos(dwg),
                'blocos': self._extrair_blocos(dwg),
                'camadas': self._extrair_camadas(dwg)
            }
            
            # Interpretar quantitativos
            quantitativos = self._interpretar_quantitativos(dados_extraidos)
            
            return {
                'status': 'sucesso',
                'arquivo': dwg_path.name,
                'dados_brutos': dados_extraidos,
                'quantitativos': quantitativos
            }
            
        except Exception as e:
            print(f"   ❌ Erro no processamento DWG: {e}")
            return {'status': 'erro', 'erro': str(e)}
    
    def _extrair_metadados(self, dwg):
        """Extrai metadados do arquivo DWG"""
        return {
            'versao_dxf': getattr(dwg, 'dxfversion', 'Desconhecida'),
            'numero_entidades': len(dwg.entities),
            'camadas': len(dwg.layers),
            'blocos': len(dwg.blocks)
        }
    
    def _extrair_entidades(self, dwg):
        """Extrai entidades relevantes do DWG"""
        entidades = []
        
        for entity in dwg.entities:
            if entity.dxftype in self.entidades_relevantes:
                entidade_data = {
                    'tipo': entity.dxftype,
                    'camada': getattr(entity, 'layer', '0'),
                    'cor': getattr(entity, 'color', 0)
                }
                
                # Dados específicos por tipo
                if entity.dxftype in ['TEXT', 'MTEXT']:
                    entidade_data['texto'] = getattr(entity, 'text', '')
                elif entity.dxftype == 'INSERT':
                    entidade_data['nome_bloco'] = getattr(entity, 'name', '')
                
                entidades.append(entidade_data)
        
        return entidades
    
    def _extrair_textos(self, dwg):
        """Extrai todos os textos do DWG"""
        textos = []
        
        for entity in dwg.entities:
            if entity.dxftype in ['TEXT', 'MTEXT']:
                texto = getattr(entity, 'text', '').strip()
                if texto and len(texto) > 2:  # Filtrar textos muito curtos
                    textos.append({
                        'texto': texto,
                        'camada': getattr(entity, 'layer', '0'),
                        'tipo': entity.dxftype
                    })
        
        return textos
    
    def _extrair_blocos(self, dwg):
        """Extrai informações sobre blocos"""
        blocos = []
        
        for block in dwg.blocks:
            blocos.append({
                'nome': block.name,
                'numero_entidades': len(block.entities)
            })
        
        return blocos
    
    def _extrair_camadas(self, dwg):
        """Extrai informações sobre camadas"""
        camadas = []
        
        for layer in dwg.layers:
            camadas.append({
                'nome': layer.name,
                'cor': layer.color,
                'visivel': not layer.off
            })
        
        return camadas
    
    def _interpretar_quantitativos(self, dados):
        """Interpreta dados extraídos para gerar quantitativos"""
        
        # Contagem de entidades por tipo
        contagem_entidades = Counter([e['tipo'] for e in dados['entidades']])
        
        # Extrair materiais dos textos
        materiais = self._extrair_materiais_textos(dados['textos'])
        
        # Analisar blocos (mobiliário, equipamentos)
        mobiliario = self._analisar_mobiliario(dados['blocos'], dados['entidades'])
        
        return {
            'estatisticas_entidades': dict(contagem_entidades),
            'total_entidades': len(dados['entidades']),
            'total_textos': len(dados['textos']),
            'materiais_encontrados': materiais,
            'mobiliario_equipamentos': mobiliario,
            'numero_camadas': len(dados['camadas']),
            'numero_blocos': len(dados['blocos'])
        }
    
    def _extrair_materiais_textos(self, textos):
        """Extrai materiais dos textos identificados"""
        materiais = []
        
        for texto_info in textos:
            texto = texto_info['texto']
            
            # Procurar por padrões de materiais
            for padrao_nome, padrao_regex in self.padroes_materiais.items():
                matches = re.finditer(padrao_regex, texto, re.IGNORECASE)
                for match in matches:
                    material = {
                        'texto_original': texto,
                        'padrao': padrao_nome,
                        'match': match.group(),
                        'camada': texto_info['camada']
                    }
                    materiais.append(material)
        
        return materiais
    
    def _analisar_mobiliario(self, blocos, entidades):
        """Analisa blocos para identificar mobiliário e equipamentos"""
        mobiliario = []
        
        # Palavras-chave para mobiliário
        keywords_mobiliario = [
            'cadeira', 'mesa', 'armario', 'estante', 'sofá', 'poltrona',
            'guarda-roupa', 'pia', 'vaso', 'chuveiro', 'geladeira',
            'fogão', 'microondas', 'lavatorio'
        ]
        
        # Verificar nomes de blocos
        for bloco in blocos:
            nome_bloco = bloco['nome'].lower()
            for keyword in keywords_mobiliario:
                if keyword in nome_bloco:
                    mobiliario.append({
                        'tipo': 'bloco',
                        'nome': bloco['nome'],
                        'categoria': keyword,
                        'entidades': bloco['numero_entidades']
                    })
                    break
        
        return mobiliario

print("✅ Processador DWG implementado e pronto para uso!")

✅ Processador DWG implementado e pronto para uso!


### BLOCO 4: PROCESSADOR RVT (REVIT)

In [51]:
# =============================================================================
# 4. PROCESSADOR RVT (REVIT) - MOCK IMPLEMENTATION
# =============================================================================

class ProcessadorRVT:
    """Processador para arquivos RVT (Revit) - Implementação Mock"""
    
    def __init__(self):
        self.categorias_relevantes = [
            'Walls', 'Doors', 'Windows', 'Floors', 'Columns',
            'Furniture', 'Plumbing Fixtures', 'Electrical Fixtures',
            'Roofs', 'Stairs', 'Rails'
        ]
    
    def extrair_dados(self, rvt_path):
        """Extrai dados de arquivo RVT (Mock para desenvolvimento)"""
        print(f"   🔧 Processando RVT (MOCK): {rvt_path.name}")
        
        if not RVT_SUPORTADO:
            # Retornar dados mock para desenvolvimento
            return self._dados_mock_rvt(rvt_path)
        
        try:
            # Implementação real viria aqui
            return self._via_revit_api(rvt_path)
        except Exception as e:
            print(f"   ❌ Erro no processamento RVT: {e}")
            return {'status': 'erro', 'erro': str(e)}
    
    def _dados_mock_rvt(self, rvt_path):
        """Gera dados mock para desenvolvimento e testes"""
        
        # Dados de exemplo baseados em projetos reais
        elementos_mock = [
            {'categoria': 'Walls', 'familia': 'Parede Interna', 'tipo': '15cm', 'quantidade': 45},
            {'categoria': 'Doors', 'familia': 'Porta Madeira', 'tipo': '0.80x2.10m', 'quantidade': 12},
            {'categoria': 'Windows', 'familia': 'Janela Alumínio', 'tipo': '1.20x1.20m', 'quantidade': 8},
            {'categoria': 'Floors', 'familia': 'Piso Cerâmico', 'tipo': 'Cerâmica 60x60', 'quantidade': 1},
            {'categoria': 'Furniture', 'familia': 'Mesa Escritório', 'tipo': 'M-001', 'quantidade': 6},
            {'categoria': 'Furniture', 'familia': 'Cadeira Giratória', 'tipo': 'C-205', 'quantidade': 12},
            {'categoria': 'Plumbing Fixtures', 'familia': 'Pia Banheiro', 'tipo': 'Pia Inox', 'quantidade': 4},
            {'categoria': 'Electrical Fixtures', 'familia': 'Tomada', 'tipo': '2P+T', 'quantidade': 28}
        ]
        
        # Calcular áreas totais (mock)
        areas_mock = {
            'area_total': 125.5,
            'area_pisos': 98.7,
            'area_paredes': 345.2,
            'area_janelas': 15.8
        }
        
        return {
            'status': 'sucesso_mock',
            'arquivo': rvt_path.name,
            'elementos': elementos_mock,
            'areas': areas_mock,
            'estatisticas': {
                'total_elementos': sum(e['quantidade'] for e in elementos_mock),
                'categorias_unicas': len(set(e['categoria'] for e in elementos_mock)),
                'familias_unicas': len(set(e['familia'] for e in elementos_mock))
            },
            'observacao': 'Dados mock - implementar Revit API para dados reais'
        }
    
    def _via_revit_api(self, rvt_path):
        """Implementação real usando Revit API"""
        # Placeholder para implementação futura
        return {
            'status': 'nao_implementado',
            'mensagem': 'Implementação Revit API requer configuração adicional'
        }

print("✅ Processador RVT (mock) implementado!")

✅ Processador RVT (mock) implementado!


### BLOCO 5: PIPELINE MULTIFORMATO UNIFICADO

In [52]:
# =============================================================================
# 5. PIPELINE MULTIFORMATO UNIFICADO
# =============================================================================

class PipelineMultiformato:
    """Pipeline unificado para processamento de múltiplos formatos"""
    
    def __init__(self):
        self.processadores = {
            '.pdf': self._processar_pdf,
            '.dwg': ProcessadorDWG().extrair_dados,
            '.rvt': ProcessadorRVT().extrair_dados
        }
        
        # Reutilizar parser do pipeline anterior
        from PyPDF2 import PdfReader
        self.pdf_reader = PdfReader
    
    def _processar_pdf(self, pdf_path):
        """Processador PDF básico - pode ser expandido"""
        print(f"   🔧 Processando PDF: {pdf_path.name}")
        
        try:
            reader = self.pdf_reader(pdf_path)
            texto = ""
            
            for pagina in reader.pages:
                texto_pagina = pagina.extract_text()
                if texto_pagina:
                    texto += texto_pagina + "\n"
            
            # Análise básica do texto
            palavras = texto.split()
            linhas = texto.split('\n')
            
            return {
                'status': 'sucesso',
                'arquivo': pdf_path.name,
                'estatisticas': {
                    'paginas': len(reader.pages),
                    'palavras': len(palavras),
                    'linhas': len(linhas),
                    'caracteres': len(texto)
                },
                'texto_amostra': texto[:500] + "..." if len(texto) > 500 else texto
            }
            
        except Exception as e:
            return {'status': 'erro', 'erro': str(e)}
    
    def processar_arquivo(self, arquivo_path):
        """Processa um arquivo baseado em sua extensão"""
        extensao = arquivo_path.suffix.lower()
        
        if extensao not in self.processadores:
            return {
                'status': 'erro',
                'erro': f'Formato não suportado: {extensao}'
            }
        
        processador = self.processadores[extensao]
        
        if extensao == '.dwg' and not DWG_SUPORTADO:
            return {'status': 'erro', 'erro': 'Dependências DWG não disponíveis'}
        
        return processador(arquivo_path)
    
    def executar_lote(self, lista_arquivos, max_arquivos=None):
        """Executa processamento em lote de múltiplos arquivos"""
        print(f"🏭 INICIANDO PROCESSAMENTO MULTIFORMATO")
        print(f"📁 Total de arquivos: {len(lista_arquivos)}")
        
        if max_arquivos:
            lista_arquivos = lista_arquivos[:max_arquivos]
            print(f"🔢 Limitando a: {max_arquivos} arquivos")
        
        # Análise de formatos
        formatos = Counter([arq.suffix.lower() for arq in lista_arquivos])
        print(f"📊 Distribuição por formato:")
        for formato, quantidade in formatos.items():
            print(f"   • {formato}: {quantidade} arquivos")
        
        resultados = {}
        
        for i, arquivo_path in enumerate(lista_arquivos, 1):
            print(f"\n[{i}/{len(lista_arquivos)}] Processando: {arquivo_path.name}")
            
            try:
                resultado = self.processar_arquivo(arquivo_path)
                resultados[arquivo_path.name] = resultado
                
                # Log de status
                if resultado.get('status', '').startswith('sucesso'):
                    print(f"   ✅ {resultado['status']}")
                else:
                    print(f"   ❌ {resultado.get('erro', 'Erro desconhecido')}")
                    
            except Exception as e:
                print(f"   💥 Erro crítico: {e}")
                resultados[arquivo_path.name] = {
                    'status': 'erro_critico',
                    'erro': str(e)
                }
        
        # Gerar relatório consolidado
        self._gerar_relatorio_consolidado(resultados)
        
        return resultados
    
    def _gerar_relatorio_consolidado(self, resultados):
        """Gera relatório consolidado do processamento"""
        print(f"\n" + "="*60)
        print("📊 RELATÓRIO CONSOLIDADO - MULTIFORMATO")
        print("="*60)
        
        total_arquivos = len(resultados)
        arquivos_sucesso = [r for r in resultados.values() if r.get('status', '').startswith('sucesso')]
        arquivos_erro = [r for r in resultados.values() if not r.get('status', '').startswith('sucesso')]
        
        print(f"📁 Total processado: {total_arquivos} arquivos")
        print(f"✅ Com sucesso: {len(arquivos_sucesso)} arquivos")
        print(f"❌ Com erro: {len(arquivos_erro)} arquivos")
        
        # Estatísticas por formato
        if arquivos_sucesso:
            print(f"\n🏗️  ESTATÍSTICAS POR FORMATO:")
            
            # Agrupar por formato e status
            formatos_info = {}
            for nome_arquivo, resultado in resultados.items():
                formato = Path(nome_arquivo).suffix.lower()
                if formato not in formatos_info:
                    formatos_info[formato] = {'sucesso': 0, 'erro': 0}
                
                if resultado.get('status', '').startswith('sucesso'):
                    formatos_info[formato]['sucesso'] += 1
                else:
                    formatos_info[formato]['erro'] += 1
            
            for formato, info in formatos_info.items():
                total_formato = info['sucesso'] + info['erro']
                taxa_sucesso = (info['sucesso'] / total_formato) * 100 if total_formato > 0 else 0
                print(f"   • {formato}: {info['sucesso']}/{total_formato} ({taxa_sucesso:.1f}% sucesso)")
        
        # Salvar resultados
        self._salvar_resultados(resultados)
    
    def _salvar_resultados(self, resultados):
        """Salva resultados em arquivo JSON"""
        try:
            pasta_processed = Path("../data/processed")
            pasta_processed.mkdir(parents=True, exist_ok=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            arquivo_json = pasta_processed / f"resultados_multiformato_{timestamp}.json"
            
            with open(arquivo_json, 'w', encoding='utf-8') as f:
                json.dump(resultados, f, indent=2, ensure_ascii=False)
            
            print(f"💾 Resultados salvos em: {arquivo_json}")
            
        except Exception as e:
            print(f"⚠️  Erro ao salvar resultados: {e}")

print("✅ Pipeline multiformato unificado implementado!")

✅ Pipeline multiformato unificado implementado!


### BLOCO 6: TESTES E DEMONSTRAÇÃO

In [53]:
# =============================================================================
# 6. TESTES E DEMONSTRAÇÃO (VERSÃO CORRIGIDA)
# =============================================================================

print("🧪 TESTES DO PIPELINE MULTIFORMATO")
print("=" * 35)

# Buscar arquivos para teste - VERSÃO CORRIGIDA
def buscar_arquivos_teste():
    """Busca arquivos nos formatos suportados para teste - VERSÃO CORRIGIDA"""
    
    print("\n🔍 BUSCANDO ARQUIVOS PARA TESTE...")
    
    # Caminho base corrigido
    caminho_base = Path(r"C:\Users\User\OneDrive - Questindustries\Projetos Python\Plataforma Quantitativos\data\raw")
    
    print(f"📁 Caminho base: {caminho_base}")
    print(f"   ✅ Existe: {caminho_base.exists()}")
    
    if not caminho_base.exists():
        print("❌ Pasta base não encontrada!")
        return []
    
    # Criar pastas específicas se não existirem
    pastas_formatos = {
        'pdf': caminho_base / 'pdf',
        'dwg': caminho_base / 'dwg', 
        'rvt': caminho_base / 'rvt'
    }
    
    print(f"\n📂 ESTRUTURA DE PASTAS:")
    for formato, pasta in pastas_formatos.items():
        pasta.mkdir(exist_ok=True)  # Cria pasta se não existir
        print(f"   📁 {pasta.name}/ - Existe: {pasta.exists()} - Arquivos: {len(list(pasta.glob('*')))}")
    
    # Buscar arquivos em todas as pastas específicas
    arquivos_encontrados = []
    formatos_suportados = ['.pdf', '.dwg', '.rvt']
    
    print(f"\n📊 ARQUIVOS ENCONTRADOS POR FORMATO:")
    
    for formato in formatos_suportados:
        # Buscar na pasta específica do formato
        pasta_formato = pastas_formatos.get(formato.replace('.', ''), caminho_base)
        arquivos_formato = list(pasta_formato.glob(f"*{formato}"))
        
        # Também buscar no diretório raiz como fallback
        if not arquivos_formato:
            arquivos_formato = list(caminho_base.glob(f"*{formato}"))
        
        arquivos_encontrados.extend(arquivos_formato)
        
        # Estatísticas detalhadas
        if arquivos_formato:
            print(f"   ✅ {formato.upper():4}: {len(arquivos_formato):2} arquivos")
            # Mostrar primeiros 3 arquivos como amostra
            for i, arquivo in enumerate(arquivos_formato[:3]):
                print(f"        {i+1}. {arquivo.name}")
            if len(arquivos_formato) > 3:
                print(f"        ... e mais {len(arquivos_formato) - 3} arquivos")
        else:
            print(f"   ❌ {formato.upper():4}: 0 arquivos")
    
    # Resumo final
    print(f"\n📈 RESUMO DA BUSCA:")
    print(f"   • Total de arquivos encontrados: {len(arquivos_encontrados)}")
    
    # Distribuição por formato
    distribuicao = Counter([arq.suffix.lower() for arq in arquivos_encontrados])
    for formato, quantidade in distribuicao.items():
        print(f"   • {formato.upper()}: {quantidade} arquivos")
    
    # Recomendações baseadas nos resultados
    if not arquivos_encontrados:
        print(f"\n⚠️  NENHUM ARQUIVO ENCONTRADO!")
        print("💡 Para testar o pipeline:")
        print("   1. Adicione arquivos PDF em: data/raw/pdf/")
        print("   2. Adicione arquivos DWG em: data/raw/dwg/") 
        print("   3. Adicione arquivos RVT em: data/raw/rvt/")
    elif all(formato == '.pdf' for formato in distribuicao.keys()):
        print(f"\n⚠️  APENAS ARQUIVOS PDF ENCONTRADOS")
        print("💡 Para testar DWG/RVT, adicione arquivos nas pastas correspondentes")
    else:
        print(f"\n✅ MÚLTIPLOS FORMATOS ENCONTRADOS - Pipeline pronto para teste!")
    
    return arquivos_encontrados

# Executar busca corrigida
arquivos_teste = buscar_arquivos_teste()

# Mostrar estatísticas para debug
if arquivos_teste:
    print(f"\n🔍 DETALHES DOS ARQUIVOS PARA PROCESSAMENTO:")
    for i, arquivo in enumerate(arquivos_teste[:5], 1):  # Mostrar apenas 5 primeiros
        print(f"   {i}. {arquivo.name} → {arquivo.suffix.upper()} → {arquivo.parent.name}/")
    
    if len(arquivos_teste) > 5:
        print(f"   ... e mais {len(arquivos_teste) - 5} arquivos")
else:
    print(f"\n❌ Nenhum arquivo encontrado para teste")

🧪 TESTES DO PIPELINE MULTIFORMATO

🔍 BUSCANDO ARQUIVOS PARA TESTE...
📁 Caminho base: C:\Users\User\OneDrive - Questindustries\Projetos Python\Plataforma Quantitativos\data\raw
   ✅ Existe: True

📂 ESTRUTURA DE PASTAS:
   📁 pdf/ - Existe: True - Arquivos: 16
   📁 dwg/ - Existe: True - Arquivos: 4
   📁 rvt/ - Existe: True - Arquivos: 0

📊 ARQUIVOS ENCONTRADOS POR FORMATO:
   ✅ .PDF: 16 arquivos
        1. Projeto arquitetura Legal Cartão Joinville.pdf
        2. PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf
        3. PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf
        ... e mais 13 arquivos
   ✅ .DWG:  4 arquivos
        1. 01_PROJETO_RESIDENCIAL_SIMPLES.dwg
        2. 02_PROJETO_COMERCIAL_ESCRITORIO.dwg
        3. 03_PROJETO_ARQUITETONICO_COMPLETO.dwg
        ... e mais 1 arquivos
   ❌ .RVT: 0 arquivos

📈 RESUMO DA BUSCA:
   • Total de arquivos encontrados: 20
   • .PDF: 16 arquivos
   • .DWG: 4 arquivos

✅ MÚLTIPLOS FORMATOS ENCONTRADOS - Pipeline pronto para teste!

🔍 D

### 🔧 BLOCO EXTRA: VERIFICAR ESTRUTURA DE PASTAS

In [54]:
# =============================================================================
# BLOCO EXTRA: VERIFICAR E CRIAR ESTRUTURA DE PASTAS
# =============================================================================

print("\n🔧 VERIFICANDO ESTRUTURA DE PASTAS...")
print("=" * 40)

def verificar_estrutura_pastas():
    """Verifica e cria a estrutura de pastas necessária"""
    
    caminho_base = Path(r"C:\Users\User\OneDrive - Questindustries\Projetos Python\Plataforma Quantitativos\data\raw")
    
    pastas_necessarias = ['pdf', 'dwg', 'rvt', 'processed']
    
    print(f"📁 Estrutura base: {caminho_base}")
    print(f"   ✅ Existe: {caminho_base.exists()}")
    
    if not caminho_base.exists():
        print("❌ Pasta base não existe! Criando...")
        caminho_base.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📂 PASTAS ESPECÍFICAS:")
    for pasta in pastas_necessarias:
        caminho_pasta = caminho_base / pasta
        caminho_pasta.mkdir(exist_ok=True)
        arquivos = list(caminho_pasta.glob('*'))
        print(f"   📁 {pasta}/ - Existe: {caminho_pasta.exists()} - Arquivos: {len(arquivos)}")
        
        # Mostrar alguns arquivos se existirem
        if arquivos:
            for i, arquivo in enumerate(arquivos[:2]):
                print(f"        {i+1}. {arquivo.name}")
            if len(arquivos) > 2:
                print(f"        ... e mais {len(arquivos) - 2} arquivos")

# Executar verificação
verificar_estrutura_pastas()


🔧 VERIFICANDO ESTRUTURA DE PASTAS...
📁 Estrutura base: C:\Users\User\OneDrive - Questindustries\Projetos Python\Plataforma Quantitativos\data\raw
   ✅ Existe: True

📂 PASTAS ESPECÍFICAS:
   📁 pdf/ - Existe: True - Arquivos: 16
        1. Projeto arquitetura Legal Cartão Joinville.pdf
        2. PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf
        ... e mais 14 arquivos
   📁 dwg/ - Existe: True - Arquivos: 4
        1. 01_PROJETO_RESIDENCIAL_SIMPLES.dwg
        2. 02_PROJETO_COMERCIAL_ESCRITORIO.dwg
        ... e mais 2 arquivos
   📁 rvt/ - Existe: True - Arquivos: 0
   📁 processed/ - Existe: True - Arquivos: 0


### BLOCO 7: EXECUÇÃO DA DEMONSTRAÇÃO

In [59]:
# =============================================================================
# 7. EXECUÇÃO DA DEMONSTRAÇÃO
# =============================================================================

print("\n🎯 DEMONSTRAÇÃO DO PIPELINE MULTIFORMATO")
print("=" * 40)

if arquivos_teste:
    # Instanciar pipeline
    pipeline_multi = PipelineMultiformato()
    
    # Mostrar quais arquivos serão processados
    print(f"\n📋 ARQUIVOS SELECIONADOS PARA DEMONSTRAÇÃO:")
    arquivos_demo = arquivos_teste[:3]  # Limitar a 3 arquivos para demo
    for i, arquivo in enumerate(arquivos_demo, 1):
        print(f"   {i}. {arquivo.name} ({arquivo.suffix.upper()})")
    
    # Executar demonstração
    resultados_demo = pipeline_multi.executar_lote(arquivos_demo)
    
    print(f"\n✅ DEMONSTRAÇÃO CONCLUÍDA!")
    
    # Análise detalhada dos resultados
    if resultados_demo:
        print(f"\n📊 ANÁLISE DOS RESULTADOS:")
        
        sucessos = [r for r in resultados_demo.values() if r.get('status', '').startswith('sucesso')]
        erros = [r for r in resultados_demo.values() if not r.get('status', '').startswith('sucesso')]
        
        print(f"   • Processados: {len(resultados_demo)} arquivos")
        print(f"   • Sucessos: {len(sucessos)}")
        print(f"   • Erros: {len(erros)}")
        
        # Mostrar amostra do primeiro resultado bem-sucedido
        if sucessos:
            primeiro_sucesso = sucessos[0]
            print(f"\n📄 AMOSTRA DO PRIMEIRO RESULTADO BEM-SUCEDIDO:")
            print(f"   Arquivo: {primeiro_sucesso.get('arquivo', 'N/A')}")
            print(f"   Status: {primeiro_sucesso.get('status', 'N/A')}")
            
            if 'estatisticas' in primeiro_sucesso:
                stats = primeiro_sucesso['estatisticas']
                print(f"   📈 Estatísticas:")
                for chave, valor in stats.items():
                    print(f"      - {chave}: {valor}")
            
            # Para DWG, mostrar quantitativos extraídos
            if 'quantitativos' in primeiro_sucesso:
                quant = primeiro_sucesso['quantitativos']
                print(f"   🏗️  Quantitativos extraídos:")
                print(f"      - Total entidades: {quant.get('total_entidades', 0)}")
                print(f"      - Total textos: {quant.get('total_textos', 0)}")
                print(f"      - Materiais encontrados: {len(quant.get('materiais_encontrados', []))}")
        
        # Mostrar erros se houver
        if erros:
            print(f"\n⚠️  DETALHES DOS ERROS:")
            for nome_arquivo, resultado in resultados_demo.items():
                if not resultado.get('status', '').startswith('sucesso'):
                    print(f"   • {nome_arquivo}: {resultado.get('erro', 'Erro desconhecido')}")
                    
else:
    print("❌ Nenhum arquivo disponível para teste")
    print("\n💡 SOLUÇÃO:")
    print("   1. Execute o script de criação de DWGs de exemplo")
    print("   2. Adicione arquivos PDF em: data/raw/pdf/")
    print("   3. Adicione arquivos DWG em: data/raw/dwg/")
    print("   4. Adicione arquivos RVT em: data/raw/rvt/")


🎯 DEMONSTRAÇÃO DO PIPELINE MULTIFORMATO

📋 ARQUIVOS SELECIONADOS PARA DEMONSTRAÇÃO:
   1. Projeto arquitetura Legal Cartão Joinville.pdf (.PDF)
   2. PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf (.PDF)
   3. PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf (.PDF)
🏭 INICIANDO PROCESSAMENTO MULTIFORMATO
📁 Total de arquivos: 3
📊 Distribuição por formato:
   • .pdf: 3 arquivos

[1/3] Processando: Projeto arquitetura Legal Cartão Joinville.pdf
   🔧 Processando PDF: Projeto arquitetura Legal Cartão Joinville.pdf


Multiple definitions in dictionary at byte 0x674b3 for key /PageMode


   ✅ sucesso

[2/3] Processando: PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf
   🔧 Processando PDF: PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf
   ✅ sucesso

[3/3] Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf
   🔧 Processando PDF: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf
   ✅ sucesso

📊 RELATÓRIO CONSOLIDADO - MULTIFORMATO
📁 Total processado: 3 arquivos
✅ Com sucesso: 3 arquivos
❌ Com erro: 0 arquivos

🏗️  ESTATÍSTICAS POR FORMATO:
   • .pdf: 3/3 (100.0% sucesso)
💾 Resultados salvos em: ..\data\processed\resultados_multiformato_20251021_130712.json

✅ DEMONSTRAÇÃO CONCLUÍDA!

📊 ANÁLISE DOS RESULTADOS:
   • Processados: 3 arquivos
   • Sucessos: 3
   • Erros: 0

📄 AMOSTRA DO PRIMEIRO RESULTADO BEM-SUCEDIDO:
   Arquivo: Projeto arquitetura Legal Cartão Joinville.pdf
   Status: sucesso
   📈 Estatísticas:
      - paginas: 3
      - palavras: 910
      - linhas: 243
      - caracteres: 6070


### BLOCO 8: ROADMAP E PRÓXIMOS PASSOS

In [56]:
# =============================================================================
# 8. ROADMAP E PRÓXIMOS PASSOS
# =============================================================================

print("\n🚀 ROADMAP DE IMPLEMENTAÇÃO MULTIFORMATO")
print("=" * 41)

print("""
📋 PRÓXIMOS PASSOS RECOMENDADOS:

1. 📦 INSTALAÇÃO DE DEPENDÊNCIAS
   • pip install dxfgrabber ezdxf
   • pip install pythonnet
   • Configurar Revit API (se aplicável)

2. 🗂️  ORGANIZAÇÃO DE ARQUIVOS
   • Criar pasta data/raw/dwg/
   • Criar pasta data/raw/rvt/
   • Adicionar arquivos de exemplo

3. 🔧 IMPLEMENTAÇÃO INCREMENTAL
   • Testar processador DWG com arquivos simples
   • Validar extração de entidades
   • Implementar parser de materiais para DWG
   • Configurar Revit API para RVT

4. 🧪 TESTES E VALIDAÇÃO
   • Comparar resultados entre formatos
   • Validar qualidade da extração
   • Ajustar padrões e algoritmos

5. 🏗️  MIGRAÇÃO PARA PRODUÇÃO
   • Integrar com pipeline existente
   • Otimizar performance
   • Documentar processos

💡 ESTADO ATUAL:
   ✅ Pipeline multiformato estruturado
   ✅ Processador PDF funcional
   🔄 Processador DWG (aguardando dependências)
   🔄 Processador RVT (mock implementado)
   ✅ Sistema de relatórios
   ✅ Gestão de erros

🎯 META: Processar DWG e RVT com mesma qualidade do PDF
""")


🚀 ROADMAP DE IMPLEMENTAÇÃO MULTIFORMATO

📋 PRÓXIMOS PASSOS RECOMENDADOS:

1. 📦 INSTALAÇÃO DE DEPENDÊNCIAS
   • pip install dxfgrabber ezdxf
   • pip install pythonnet
   • Configurar Revit API (se aplicável)

2. 🗂️  ORGANIZAÇÃO DE ARQUIVOS
   • Criar pasta data/raw/dwg/
   • Criar pasta data/raw/rvt/
   • Adicionar arquivos de exemplo

3. 🔧 IMPLEMENTAÇÃO INCREMENTAL
   • Testar processador DWG com arquivos simples
   • Validar extração de entidades
   • Implementar parser de materiais para DWG
   • Configurar Revit API para RVT

4. 🧪 TESTES E VALIDAÇÃO
   • Comparar resultados entre formatos
   • Validar qualidade da extração
   • Ajustar padrões e algoritmos

5. 🏗️  MIGRAÇÃO PARA PRODUÇÃO
   • Integrar com pipeline existente
   • Otimizar performance
   • Documentar processos

💡 ESTADO ATUAL:
   ✅ Pipeline multiformato estruturado
   ✅ Processador PDF funcional
   🔄 Processador DWG (aguardando dependências)
   🔄 Processador RVT (mock implementado)
   ✅ Sistema de relatórios
   ✅ Gest

## 📁 CÓDIGOS EXTRAS

### 📁 CRIANDO ARQUIVOS DWG DE EXEMPLO

In [58]:
# 🔧 CORREÇÃO PARA CRIAR ARQUIVOS DWG DE EXEMPLO

print("🛠️ CRIANDO ARQUIVOS DWG DE EXEMPLO...")
print("=" * 55)

import os
from pathlib import Path
import ezdxf
from ezdxf import units
import random
from datetime import datetime

def criar_pasta_dwg():
    """Cria pasta para arquivos DWG de exemplo"""
    pasta_dwg = Path("data/raw/dwg")
    pasta_dwg.mkdir(parents=True, exist_ok=True)
    print(f"📁 Pasta criada: {pasta_dwg}")
    return pasta_dwg

def criar_dwg_projeto_residencial_simples():
    """Cria DWG de projeto residencial simples - VERSÃO CORRIGIDA"""
    doc = ezdxf.new('R2010')
    doc.units = units.M
    
    # Configurar layers
    doc.layers.new('PAREDES', dxfattribs={'color': 7})
    doc.layers.new('PORTAS', dxfattribs={'color': 1})
    doc.layers.new('JANELAS', dxfattribs={'color': 4})
    doc.layers.new('TEXTOS', dxfattribs={'color': 3})
    doc.layers.new('MOBILIARIO', dxfattribs={'color': 2})
    
    msp = doc.modelspace()
    
    # CORREÇÃO: Usar insert em vez de set_pos()
    msp.add_text('PROJETO RESIDENCIAL - CASA 001', 
                 dxfattribs={
                     'layer': 'TEXTOS',
                     'height': 0.3
                 }).set_placement((2, 10))  # CORREÇÃO AQUI
    
    # Desenhar paredes (retângulo simples)
    pontos_paredes = [(0, 0), (8, 0), (8, 6), (0, 6), (0, 0)]
    msp.add_lwpolyline(pontos_paredes, dxfattribs={'layer': 'PAREDES'})
    
    # Adicionar portas
    msp.add_line((0, 1), (1, 1), dxfattribs={'layer': 'PORTAS'})
    msp.add_line((4, 0), (4, 1), dxfattribs={'layer': 'PORTAS'})
    
    # Adicionar janelas
    msp.add_line((2, 6), (3, 6), dxfattribs={'layer': 'JANELAS'})
    msp.add_line((5, 6), (6, 6), dxfattribs={'layer': 'JANELAS'})
    
    # CORREÇÃO: Todos os textos com set_placement()
    materiais = [
        (1, 7, "M-001 - Porta Madeira 0.80x2.10m"),
        (1, 6.5, "M-002 - Janela Alumínio 1.00x1.20m"),
        (1, 6, "M-003 - Piso Cerâmico 60x60cm"),
        (1, 5.5, "M-004 - Parede Drywall 15cm"),
        (1, 5, "Qtd: 12 un - Tijolo Baiano"),
        (1, 4.5, "M-005 - Tomada 2P+T - 28 un"),
        (1, 4, "M-006 - Lâmpada LED - 15 un"),
        (1, 3.5, "M-007 - Pia Inox - 2 un"),
        (1, 3, "M-008 - Vaso Sanitário - 2 un"),
        (1, 2.5, "M-009 - Chuveiro Elétrico - 2 un")
    ]
    
    for x, y, texto in materiais:
        msp.add_text(texto, dxfattribs={
            'layer': 'TEXTOS',
            'height': 0.15
        }).set_placement((x, y))  # CORREÇÃO AQUI
    
    # Adicionar mobiliário
    mobiliario = [
        (2, 2, "Cadeira"),
        (6, 2, "Mesa"),
        (2, 4, "Sofá"),
        (6, 4, "Armário")
    ]
    
    for x, y, nome in mobiliario:
        msp.add_text(nome, dxfattribs={
            'layer': 'MOBILIARIO',
            'height': 0.12
        }).set_placement((x, y))  # CORREÇÃO AQUI
        
        pontos = [(x-0.3, y-0.1), (x+0.3, y-0.1), (x+0.3, y+0.1), (x-0.3, y+0.1), (x-0.3, y-0.1)]
        msp.add_lwpolyline(pontos, dxfattribs={'layer': 'MOBILIARIO'})
    
    return doc

def criar_dwg_projeto_comercial():
    """Cria DWG de projeto comercial/escritório - VERSÃO CORRIGIDA"""
    doc = ezdxf.new('R2010')
    doc.units = units.M
    
    doc.layers.new('DIVISORIAS', dxfattribs={'color': 5})
    doc.layers.new('ILUMINACAO', dxfattribs={'color': 6})
    doc.layers.new('AR_CONDICIONADO', dxfattribs={'color': 8})
    doc.layers.new('HIDRAULICA', dxfattribs={'color': 4})
    doc.layers.new('TEXTOS', dxfattribs={'color': 3})
    
    msp = doc.modelspace()
    
    # CORREÇÃO: set_placement()
    msp.add_text('PROJETO COMERCIAL - ESCRITÓRIO 001', 
                 dxfattribs={'layer': 'TEXTOS', 'height': 0.3}).set_placement((3, 12))
    
    pontos_principal = [(0, 0), (10, 0), (10, 8), (0, 8), (0, 0)]
    msp.add_lwpolyline(pontos_principal, dxfattribs={'layer': 'DIVISORIAS'})
    
    msp.add_line((4, 0), (4, 5), dxfattribs={'layer': 'DIVISORIAS'})
    msp.add_line((0, 5), (4, 5), dxfattribs={'layer': 'DIVISORIAS'})
    
    quantitativos = [
        (0.5, 11, "MEMORIAL DE MATERIAIS - ESCRITÓRIO"),
        (0.5, 10.5, "M-101 - Divisória Drywall 10cm - 45m²"),
        (0.5, 10, "M-102 - Piso Carpete - 80m²"),
        (0.5, 9.5, "M-103 - Forro Gesso - 80m²"),
        (0.5, 9, "M-104 - Porta de Correr - 3 un"),
        (0.5, 8.5, "M-105 - Luminária LED - 25 un"),
        (0.5, 8, "M-106 - Tomada Telecom - 40 un"),
        (0.5, 7.5, "M-107 - Split 9000BTU - 4 un"),
        (0.5, 7, "M-108 - Mesa Escritório M-001 - 8 un"),
        (0.5, 6.5, "M-109 - Cadeira Executiva C-205 - 12 un"),
        (0.5, 6, "M-110 - Armário Aço A-301 - 4 un"),
        (0.5, 5.5, "M-111 - Pia Aço Inox - 2 un")
    ]
    
    for x, y, texto in quantitativos:
        msp.add_text(texto, dxfattribs={
            'layer': 'TEXTOS',
            'height': 0.12
        }).set_placement((x, y))  # CORREÇÃO AQUI
    
    # Símbolos de instalações
    for x, y in [(2, 7), (5, 7), (8, 7), (2, 3), (5, 3), (8, 3)]:
        msp.add_circle((x, y), 0.15, dxfattribs={'layer': 'ILUMINACAO'})
    
    for x, y in [(1, 7.5), (9, 7.5)]:
        msp.add_text("AC", dxfattribs={
            'layer': 'AR_CONDICIONADO',
            'height': 0.1
        }).set_placement((x, y))  # CORREÇÃO AQUI
    
    return doc

def criar_dwg_projeto_arquitetônico_completo():
    """Cria DWG mais complexo - VERSÃO CORRIGIDA"""
    doc = ezdxf.new('R2018')
    doc.units = units.M
    
    disciplinas = {
        'ARQUITETURA': 7,
        'ELETRICA': 1,
        'HIDRAULICA': 5,
        'ESTRUTURA': 3,
        'LEGENDAS': 2
    }
    
    for nome, cor in disciplinas.items():
        doc.layers.new(nome, dxfattribs={'color': cor})
    
    msp = doc.modelspace()
    
    # CORREÇÃO: set_placement()
    msp.add_text('PROJETO ARQUITETÔNICO COMPLETO - EDIFÍCIO MULTIANDAR', 
                 dxfattribs={'layer': 'LEGENDAS', 'height': 0.25}).set_placement((2, 15))
    
    andares = [
        ("TERREO", 0, "Área: 120m²"),
        ("1º ANDAR", 4, "Área: 110m²"), 
        ("2º ANDAR", 8, "Área: 110m²"),
        ("COBERTURA", 12, "Área: 90m²")
    ]
    
    for i, (nome_andar, y_base, area) in enumerate(andares):
        pontos_andar = [(0, y_base), (8, y_base), (8, y_base+3), (0, y_base+3), (0, y_base)]
        msp.add_lwpolyline(pontos_andar, dxfattribs={'layer': 'ARQUITETURA'})
        
        msp.add_text(f"{nome_andar} - {area}", 
                    dxfattribs={'layer': 'LEGENDAS', 'height': 0.15}).set_placement((0.2, y_base+2.8))
        
        if nome_andar == "TERREO":
            elementos = [
                "M-201 - Porta Principal 1.20x2.40m",
                "M-202 - Janela Vitro 2.00x1.50m - 4un",
                "M-203 - Piso Granito - 120m²"
            ]
        elif nome_andar == "1º ANDAR":
            elementos = [
                "M-204 - Quarto Suite - 15m²",
                "M-205 - Guarda-Roupa Embutido - 3un", 
                "M-206 - Piso Porcelanato - 110m²"
            ]
        elif nome_andar == "2º ANDAR":
            elementos = [
                "M-207 - Sala Estar - 25m²",
                "M-208 - Varanda Vidro - 8m²",
                "M-209 - Piso Madeira - 85m²"
            ]
        else:
            elementos = [
                "M-210 - Área Gourmet - 40m²",
                "M-211 - Churrasqueira Alvenaria",
                "M-212 - Piso Antiderrapante - 50m²"
            ]
        
        for j, elemento in enumerate(elementos):
            msp.add_text(elemento, dxfattribs={
                'layer': 'LEGENDAS',
                'height': 0.1
            }).set_placement((8.5, y_base + 2.5 - j*0.3))
    
    tabela_quantitativos = [
        (12, 14, "TABELA DE QUANTITATIVOS CONSOLIDADA"),
        (12, 13.7, "CÓDIGO | DESCRIÇÃO | QTD | UNID"),
        (12, 13.4, "M-001  | Tijolo   | 2500| un"),
        (12, 13.1, "M-002  | Cimento  | 120 | kg"), 
        (12, 12.8, "M-003  | Areia    | 8   | m³"),
        (12, 12.5, "M-004  | Brita    | 6   | m³"),
        (12, 12.2, "M-005  | Aço CA50 | 1800| kg"),
        (12, 11.9, "M-006  | Telha    | 450 | un"),
        (12, 11.6, "M-007  | Janela   | 12  | un"),
        (12, 11.3, "M-008  | Porta    | 8   | un"),
        (12, 11.0, "M-009  | Piso     | 365 | m²"),
        (12, 10.7, "M-010  | Tinta    | 45  | L")
    ]
    
    for x, y, texto in tabela_quantitativos:
        msp.add_text(texto, dxfattribs={
            'layer': 'LEGENDAS', 
            'height': 0.08
        }).set_placement((x, y))
    
    return doc

def criar_dwg_com_medidas_e_cotagens():
    """Cria DWG com medidas - VERSÃO CORRIGIDA"""
    doc = ezdxf.new('R2010')
    doc.units = units.MM
    
    doc.layers.new('GEOMETRIA', dxfattribs={'color': 7})
    doc.layers.new('COTAS', dxfattribs={'color': 1})
    doc.layers.new('OBSERVACOES', dxfattribs={'color': 3})
    
    msp = doc.modelspace()
    
    # CORREÇÃO: set_placement()
    msp.add_text('DWG COM MEDIDAS E COTAGENS - TESTE DIMENSIONAL',
                 dxfattribs={'layer': 'OBSERVACOES', 'height': 50}).set_placement((500, 1800))
    
    # Geometria principal
    pontos = [(0, 0), (4000, 0), (4000, 3000), (0, 3000), (0, 0)]
    msp.add_lwpolyline(pontos, dxfattribs={'layer': 'GEOMETRIA'})
    
    # Cotas dimensionais
    msp.add_line((0, -100), (4000, -100), dxfattribs={'layer': 'COTAS'})
    msp.add_line((0, -150), (0, -50), dxfattribs={'layer': 'COTAS'})
    msp.add_line((4000, -150), (4000, -50), dxfattribs={'layer': 'COTAS'})
    msp.add_text('4000 mm', dxfattribs={
        'layer': 'COTAS', 'height': 30
    }).set_placement((2000, -200))
    
    msp.add_line((-100, 0), (-100, 3000), dxfattribs={'layer': 'COTAS'})
    msp.add_line((-150, 0), (-50, 0), dxfattribs={'layer': 'COTAS'})
    msp.add_line((-150, 3000), (-50, 3000), dxfattribs={'layer': 'COTAS'})
    msp.add_text('3000 mm', dxfattribs={
        'layer': 'COTAS', 'height': 30
    }).set_placement((-250, 1500))
    
    elementos = [
        (1000, 1000, "Janela 1200x1400mm"),
        (2500, 1000, "Porta 800x2100mm"), 
        (500, 2000, "Armário 1800x600x400mm"),
        (3000, 2000, "Pia 1000x500mm")
    ]
    
    for x, y, descricao in elementos:
        msp.add_text(descricao, dxfattribs={
            'layer': 'OBSERVACOES', 'height': 25
        }).set_placement((x, y))
        
        if "Janela" in descricao:
            msp.add_lwpolyline([(x, y), (x+1200, y), (x+1200, y+1400), (x, y+1400), (x, y)], 
                              dxfattribs={'layer': 'GEOMETRIA'})
        elif "Porta" in descricao:
            msp.add_lwpolyline([(x, y), (x+800, y), (x+800, y+2100), (x, y+2100), (x, y)], 
                              dxfattribs={'layer': 'GEOMETRIA'})
    
    return doc

# FUNÇÃO PRINCIPAL CORRIGIDA
def criar_todos_dwg_exemplo():
    """Cria todos os arquivos DWG de exemplo - VERSÃO CORRIGIDA"""
    
    pasta_dwg = criar_pasta_dwg()
    
    exemplos = [
        ("01_PROJETO_RESIDENCIAL_SIMPLES.dwg", criar_dwg_projeto_residencial_simples),
        ("02_PROJETO_COMERCIAL_ESCRITORIO.dwg", criar_dwg_projeto_comercial),
        ("03_PROJETO_ARQUITETONICO_COMPLETO.dwg", criar_dwg_projeto_arquitetônico_completo),
        ("04_DWG_COM_MEDIDAS_COTAGENS.dwg", criar_dwg_com_medidas_e_cotagens)
    ]
    
    print("\n🛠️ CRIANDO ARQUIVOS DWG...")
    print("-" * 30)
    
    arquivos_criados = []
    
    for nome_arquivo, funcao_criacao in exemplos:
        try:
            doc = funcao_criacao()
            caminho_arquivo = pasta_dwg / nome_arquivo
            doc.saveas(caminho_arquivo)
            print(f"✅ {nome_arquivo:45} | Criado com sucesso!")
            
            estatisticas = f"Layers: {len(doc.layers)}, Entidades: {len(doc.modelspace())}"
            print(f"   📊 {estatisticas}")
            
            arquivos_criados.append(nome_arquivo)
            
        except Exception as e:
            print(f"❌ {nome_arquivo:45} | Erro: {e}")
    
    return pasta_dwg, arquivos_criados

# EXECUTAR CRIAÇÃO CORRIGIDA
try:
    pasta_resultado, arquivos_criados = criar_todos_dwg_exemplo()
    
    print(f"\n🎉 CRIAÇÃO CONCLUÍDA!")
    print("=" * 45)
    print(f"📁 Arquivos salvos em: {pasta_resultado}")
    print(f"📊 Total: {len(arquivos_criados)} arquivos DWG criados com sucesso")
    
    if arquivos_criados:
        print(f"📋 Arquivos criados:")
        for arquivo in arquivos_criados:
            print(f"   • {arquivo}")
    
    print("\n🔧 PRÓXIMOS PASSOS:")
    print("1. Execute o pipeline multiformato para processar os DWGs")
    print("2. Verifique a extração de textos e entidades") 
    print("3. Ajuste os padrões regex conforme necessário")
    print("4. Teste com seus próprios arquivos DWG reais")
    
except Exception as e:
    print(f"❌ ERRO NA CRIAÇÃO: {e}")
    print("💡 Verifique se a biblioteca ezdxf está instalada:")
    print("   pip install ezdxf")

🛠️ CRIANDO ARQUIVOS DWG DE EXEMPLO...
📁 Pasta criada: data\raw\dwg

🛠️ CRIANDO ARQUIVOS DWG...
------------------------------
✅ 01_PROJETO_RESIDENCIAL_SIMPLES.dwg            | Criado com sucesso!
   📊 Layers: 7, Entidades: 24
✅ 02_PROJETO_COMERCIAL_ESCRITORIO.dwg           | Criado com sucesso!
   📊 Layers: 7, Entidades: 24
✅ 03_PROJETO_ARQUITETONICO_COMPLETO.dwg         | Criado com sucesso!
   📊 Layers: 7, Entidades: 33
✅ 04_DWG_COM_MEDIDAS_COTAGENS.dwg               | Criado com sucesso!
   📊 Layers: 5, Entidades: 16

🎉 CRIAÇÃO CONCLUÍDA!
📁 Arquivos salvos em: data\raw\dwg
📊 Total: 4 arquivos DWG criados com sucesso
📋 Arquivos criados:
   • 01_PROJETO_RESIDENCIAL_SIMPLES.dwg
   • 02_PROJETO_COMERCIAL_ESCRITORIO.dwg
   • 03_PROJETO_ARQUITETONICO_COMPLETO.dwg
   • 04_DWG_COM_MEDIDAS_COTAGENS.dwg

🔧 PRÓXIMOS PASSOS:
1. Execute o pipeline multiformato para processar os DWGs
2. Verifique a extração de textos e entidades
3. Ajuste os padrões regex conforme necessário
4. Teste com seus pró